In [11]:
!pip install speechbrain
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 630.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Imports

In [1]:
import os

In [12]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio
import torch

In [13]:
import librosa
import numpy as np

# from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio as sisnr
# from torchmetrics.functional.audio import signal_distortion_ratio as sdr

from torchmetrics.audio import ScaleInvariantSignalNoiseRatio as sisnr
from torchmetrics.audio import SignalDistortionRatio as sdr

from itertools import permutations

from speechbrain.nnet.losses import PitWrapper

In [14]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# Make Librimix

In [2]:
!wget -c --tries=0 --read-timeout=20 http://www.openslr.org/resources/12/test-clean.tar.gz -P "/"
!tar -xzf "test-clean.tar.gz" -C "/content/"
!rm -rf test-clean.tar.gz

--2024-04-04 04:56:50--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2024-04-04 04:56:50--  http://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

est-clean.tar.gz    100%[===================>] 330.60M  25.9MB/s    in 14s     

2024-04-04 04:57:04 (24.4 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [3]:
!wget -c --tries=0 --read-timeout=20 https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip -P "/"
!unzip -qn wham_noise.zip -d "/content/"
!rm -rf wham_noise.zip

--2024-04-04 04:57:19--  https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip
Resolving my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)... 3.5.22.183, 54.231.227.113, 52.217.133.25, ...
Connecting to my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)|3.5.22.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18194930748 (17G) [application/zip]
Saving to: ‘wham_noise.zip’

ham_noise.zip       100%[===================>]  16.95G  32.4MB/s    in 9m 53s  

2024-04-04 05:07:12 (29.2 MB/s) - ‘wham_noise.zip’ saved [18194930748/18194930748]



In [4]:
!git clone https://github.com/JorisCos/LibriMix
!pip install -r LibriMix/requirements.txt

Cloning into 'LibriMix'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 274 (delta 71), reused 58 (delta 58), pack-reused 163
Receiving objects: 100% (274/274), 33.43 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (116/116), done.
  Preparing metadata (setup.py) ... done
  Created wheel for pysndfx: filename=pysndfx-0.3.6-py3-none-any.whl size=6539 sha256=f031b4591faf42f8aa87cb9a16c66f2499641ee37c7fa091333890175c5b6f81
  Stored in directory: /root/.cache/pip/wheels/e8/73/04/275af1c6a789bbc12f388bce2d5353e77579e3aaadd36c61fe
Successfully built pysndfx


In [5]:
dir_path = "/content/LibriMix/metadata/Libri2Mix"

for file in os.listdir(dir_path):
    if "libri2mix_test-clean" not in file:
        !rm {dir_path + "/" + file}

In [6]:
metadata_dir = "LibriMix/metadata/Libri2Mix"

!mkdir librimix
!python LibriMix/scripts/create_librimix_from_metadata.py --librispeech_dir "LibriSpeech" \
    --wham_dir "/content/wham_noise" \
    --metadata_dir {metadata_dir} \
    --librimix_outdir "librimix" \
    --n_src 2 \
    --freqs 8k \
    --modes max \
    --types mix_clean mix_single

Creating mixtures and sources from LibriMix/metadata/Libri2Mix/libri2mix_test-clean.csv in librimix/Libri2Mix/wav8k/max/test
100% 3000/3000 [01:43<00:00, 28.87it/s]


In [7]:
!rm -r wham_noise
!rm -r LibriSpeech

In [8]:
import pandas as pd

csv_path1 = "/content/librimix/Libri2Mix/wav8k/max/metadata/metrics_test_mix_clean.csv"
csv_path2 = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"

dataframe1 = pd.read_csv(csv_path1)
dataframe2 = pd.read_csv(csv_path2)

In [9]:
dataframe2

,mixture_ID,mixture_path,source_1_path,source_2_path,length
0,4077-13754-0001_5142-33396-0065,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,41560
1,6930-76324-0027_5683-32879-0011,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,73680
2,1188-133604-0025_4992-23283-0016,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,59600
3,5683-32866-0012_121-121726-0001,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,66080
4,1995-1837-0025_5142-36377-0017,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,76041
...,...,...,...,...,...
2995,1580-141084-0018_4446-2273-0019,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,26760
2996,260-123288-0000_61-70968-0039,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,30440
2997,672-122797-0021_7021-79759-0003,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,36960
2998,4970-29095-0005_4446-2271-0018,/content/librimix/Libri2Mix/wav8k/max/test/mix...,/content/librimix/Libri2Mix/wav8k/max/test/s1/...,/content/librimix/Libri2Mix/wav8k/max/test/s2/...,45720


In [10]:
dataframe1

,mixture_ID,source_1_SNR,source_2_SNR
0,4077-13754-0001_5142-33396-0065,-2.769124,2.769123
1,6930-76324-0027_5683-32879-0011,0.150564,-0.150566
2,1188-133604-0025_4992-23283-0016,2.695327,-2.695328
3,5683-32866-0012_121-121726-0001,5.008483,-5.008486
4,1995-1837-0025_5142-36377-0017,1.175087,-1.175089
...,...,...,...
2995,1580-141084-0018_4446-2273-0019,2.654774,-2.654775
2996,260-123288-0000_61-70968-0039,3.495769,-3.495770
2997,672-122797-0021_7021-79759-0003,2.160409,-2.160410
2998,4970-29095-0005_4446-2271-0018,-0.479279,0.479278


# Task 2

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

def load_csv_data(csv_path):
    return pd.read_csv(csv_path)

def split_dataset(data, train_size=0.7, random_state=42):
    train_set, test_set = train_test_split(data, train_size=train_size, random_state=random_state)
    return train_set, test_set

librimix_csv_path = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"
librimix_csv = load_csv_data(librimix_csv_path)
train_set, test_set = split_dataset(librimix_csv)

In [16]:
def load_and_initialize_model(model_source, save_directory, device_name):
    # Assuming `separator` is an object or module that has been previously defined and imported.
    # This object should have a method `from_hparams` capable of loading the model.
    model = separator.from_hparams(source=model_source, savedir=save_directory, run_opts={"device": device_name})
    model = model.to(device_name)  # Ensure the model is moved to the specified device.
    return model

model_source = "speechbrain/sepformer-whamr"
save_directory = "pretrained_models/sepformer-whamr"

# Load and initialize the model
model = load_and_initialize_model(model_source, save_directory, device_name)

hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

In [17]:
def load_audio(path):
    y , sr = librosa.load(path, sr = 8000)
    return torch.tensor(y).unsqueeze(0)

In [29]:
class SI_SNR_i:

    def __init__(self,**kwargs):

        self.sisnr = sisnr(**kwargs)

    def __call__(self, mixed_signal, predicted_signals, target_signals):

        self.sisnr.to(target_signals.device)
        mixed_signal = torch.concat([mixed_signal,mixed_signal],dim=2).to(target_signals.device)

        sisnr_pred, _ = self.sisnr_with_pi(predicted_signals , target_signals)
        sisnr_base = self.sisnr(mixed_signal , target_signals)

        sisnr_i = torch.mean(sisnr_pred) - sisnr_base

        return sisnr_i

    def sisnr_with_pi(self, pred, real):
        ''' using the si-snr metric with permutation invariance (pi) '''

        losses = []
        perms  = []

        for pred, label in zip(pred, real):

            pred  = pred.unsqueeze(0)
            label = label.unsqueeze(0)

            loss, p = self.loss_mat(pred, label)

            perms.append(p)
            losses.append(loss)

        loss = torch.stack(losses)

        return loss, perms

    def loss_mat(self, preds , targets):

        n_sources = targets.shape[-1]

        loss = None
        assigned_perm = None
        for p in permutations(range(n_sources)):
            c_loss = self.sisnr(preds , self.reorder_tensor(targets,[p]))
            print(torch.sum(torch.abs(preds - self.reorder_tensor(targets,[p]))))
            if loss is None or loss < c_loss:
                loss = c_loss
                assigned_perm = p
            print(p,c_loss,loss,preds.shape)
        return loss, assigned_perm

    def reorder_tensor(self, tensor, p):

        reordered = torch.zeros_like(tensor, device=tensor.device)
        for b in range(tensor.shape[0]):
            reordered[b] = tensor[b][..., p[b]].clone()
        return reordered

metric = SI_SNR_i()

sr1 = load_audio(train_set.iloc[1].source_1_path)
sr2 = load_audio(train_set.iloc[1].source_2_path)
mix = load_audio(train_set.iloc[1].mixture_path)

output = torch.stack([sr1,sr2],dim=-1)
target = torch.stack([sr1,sr2],dim=-1)
mix    = mix.unsqueeze(-1)

print(output.shape , target.shape, mix.shape)

print(metric(mix,output,target))
# print(metric(mix,mat3,mat3))

torch.Size([1, 66720, 2]) torch.Size([1, 66720, 2]) torch.Size([1, 66720, 1])
tensor(0.)
(0, 1) tensor(24.5670) tensor(24.5670) torch.Size([1, 66720, 2])
tensor(3117.5347)
(1, 0) tensor(23.5531) tensor(24.5670) torch.Size([1, 66720, 2])
tensor(24.5670)
